# returns

In [1]:
import vectorbt as vbt

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from numba import njit, f8, i8, b1, optional
import empyrical

In [3]:
index = pd.DatetimeIndex([
    datetime(2018, 1, 1),
    datetime(2018, 1, 2),
    datetime(2018, 1, 3),
    datetime(2018, 1, 4),
    datetime(2018, 1, 5)
], freq='D')
columns = ['a', 'b', 'c']
ts = pd.DataFrame({
    'a': [1, 2, 3, 4, 5], 
    'b': [5, 4, 3, 2, 1],
    'c': [1, 2, 3, 2, 1]}, index=index).astype(np.float32)

print(ts)

              a    b    c
2018-01-01  1.0  5.0  1.0
2018-01-02  2.0  4.0  2.0
2018-01-03  3.0  3.0  3.0
2018-01-04  4.0  2.0  2.0
2018-01-05  5.0  1.0  1.0


In [4]:
big_ts = pd.DataFrame(np.random.randint(10, size=(1000, 1000)).astype(float))
big_ts.index = [datetime(2018, 1, 1) + timedelta(days=i) for i in range(1000)]
big_ts.shape

(1000, 1000)

In [5]:
returns = ts.vbt.tseries.pct_change()
print(returns)

big_returns = big_ts.vbt.tseries.pct_change()

                   a         b         c
2018-01-01       NaN       NaN       NaN
2018-01-02  1.000000 -0.200000  1.000000
2018-01-03  0.500000 -0.250000  0.500000
2018-01-04  0.333333 -0.333333 -0.333333
2018-01-05  0.250000 -0.500000 -0.500000


In [6]:
# Test year frequency
print(returns.vbt.returns.year_freq)
print(returns['a'].vbt.returns.year_freq)
print(returns.vbt.returns(year_freq='252 days').year_freq)
print(returns['a'].vbt.returns(year_freq='252 days').year_freq)

365 days 00:00:00
365 days 00:00:00
252 days 00:00:00
252 days 00:00:00


In [7]:
print(returns.vbt.returns.ann_factor) # default
print(returns.vbt.returns(year_freq='252 days').ann_factor)

365.0
252.0


In [8]:
print(returns['a'].vbt.returns.daily()) # already daily, do nothing
print(returns.vbt.returns.daily())

%timeit big_returns.vbt.returns.daily()

2018-01-01         NaN
2018-01-02    1.000000
2018-01-03    0.500000
2018-01-04    0.333333
2018-01-05    0.250000
Freq: D, Name: a, dtype: float64
                   a         b         c
2018-01-01       NaN       NaN       NaN
2018-01-02  1.000000 -0.200000  1.000000
2018-01-03  0.500000 -0.250000  0.500000
2018-01-04  0.333333 -0.333333 -0.333333
2018-01-05  0.250000 -0.500000 -0.500000
14.7 µs ± 937 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
print(returns['a'].vbt.returns.annual())
print(returns.vbt.returns.annual())

%timeit big_returns.vbt.returns.annual()

2018-01-01    4.0
Freq: 365D, Name: a, dtype: float64
              a    b             c
2018-01-01  4.0 -0.8  2.980232e-08
24.9 ms ± 5 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
print(empyrical.cum_returns(returns['a']))
%timeit empyrical.cum_returns(big_returns[0])

print(returns['a'].vbt.returns.cumulative())
%timeit big_returns[0].vbt.returns.cumulative()

print(returns.vbt.returns.cumulative())
print(returns.vbt.returns.cumulative(start_value=1))
print(returns.vbt.returns.cumulative(start_value=[1, 2, 3])) # also accepts array
%timeit big_returns.vbt.returns.cumulative()

2018-01-01    0.0
2018-01-02    1.0
2018-01-03    2.0
2018-01-04    3.0
2018-01-05    4.0
Freq: D, dtype: float64
3.13 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
2018-01-01    0.0
2018-01-02    1.0
2018-01-03    2.0
2018-01-04    3.0
2018-01-05    4.0
Freq: D, Name: a, dtype: float64
437 µs ± 72.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
              a    b             c
2018-01-01  0.0  0.0  0.000000e+00
2018-01-02  1.0 -0.2  1.000000e+00
2018-01-03  2.0 -0.4  2.000000e+00
2018-01-04  3.0 -0.6  1.000000e+00
2018-01-05  4.0 -0.8  2.980232e-08
              a    b    c
2018-01-01  1.0  1.0  1.0
2018-01-02  2.0  0.8  2.0
2018-01-03  3.0  0.6  3.0
2018-01-04  4.0  0.4  2.0
2018-01-05  5.0  0.2  1.0
              a    b    c
2018-01-01  1.0  2.0  3.0
2018-01-02  2.0  1.6  6.0
2018-01-03  3.0  1.2  9.0
2018-01-04  4.0  0.8  6.0
2018-01-05  5.0  0.4  3.0
27.1 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
print(empyrical.cum_returns_final(returns['a']))
%timeit empyrical.cum_returns_final(big_returns[0])

print(returns['a'].vbt.returns.total())
%timeit big_returns[0].vbt.returns.total()

print(returns.vbt.returns.total())
%timeit big_returns.vbt.returns.total()

4.000000149011612


/Users/olegpolakow/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


721 µs ± 142 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.000000149011612
312 µs ± 26.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a    4.000000e+00
b   -8.000000e-01
c    2.980232e-08
dtype: float64
7.06 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
print(returns['a'].vbt.returns.drawdown())
print(returns.vbt.returns.drawdown())

%timeit big_returns.vbt.returns.drawdown()

2018-01-01    0.0
2018-01-02    0.0
2018-01-03    0.0
2018-01-04    0.0
2018-01-05    0.0
Freq: D, Name: a, dtype: float64
              a    b         c
2018-01-01  0.0  0.0  0.000000
2018-01-02  0.0 -0.2  0.000000
2018-01-03  0.0 -0.4  0.000000
2018-01-04  0.0 -0.6 -0.333333
2018-01-05  0.0 -0.8 -0.666667


/Users/olegpolakow/Documents/SourceTree/vectorbt/vectorbt/tseries/accessors.py:464: RuntimeWarning: invalid value encountered in true_divide
  return self.wrap(self.to_2d_array() / nb.expanding_max_nb(self.to_2d_array()) - 1)


44.2 ms ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
print(empyrical.max_drawdown(returns['b']))
%timeit empyrical.max_drawdown(big_returns[0])

print(returns['b'].vbt.returns.max_drawdown())
%timeit big_returns[0].vbt.returns.max_drawdown()

print(returns.vbt.returns.max_drawdown())
%timeit big_returns.vbt.returns.max_drawdown()

-0.7999999910593033
206 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
0.7999999910593032
306 µs ± 23.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a    0.000000
b    0.800000
c    0.666667
dtype: float64
17.9 ms ± 396 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
vbt.defaults.returns['year_freq'] = '252 days' # same as empyrical

In [15]:
print(empyrical.annual_return(returns['a']))
%timeit empyrical.annual_return(big_returns[0])

print(returns['a'].vbt.returns.annualized_return())
%timeit big_returns[0].vbt.returns.annualized_return()

print(returns.vbt.returns.annualized_return())
%timeit big_returns.vbt.returns.annualized_return()

1.690786886567203e+35
478 µs ± 31.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.6907868865671834e+35
326 µs ± 14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a    1.690787e+35
b   -1.000000e+00
c    1.502038e-06
dtype: float64
5.41 ms ± 219 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
print(empyrical.annual_volatility(returns['a'], alpha=3.))
%timeit empyrical.annual_volatility(big_returns[0], alpha=3.)

print(returns['a'].vbt.returns.annualized_volatility(levy_alpha=3.))
%timeit big_returns[0].vbt.returns.annualized_volatility(levy_alpha=3.)

print(returns.vbt.returns.annualized_volatility(levy_alpha=3.))
print(returns.vbt.returns.annualized_volatility(levy_alpha=[1, 2, 3]))
%timeit big_returns.vbt.returns.annualized_volatility(levy_alpha=3.)

2.121838249438074
108 µs ± 8.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2.121838249438074
332 µs ± 20.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a    2.121838
b    0.830587
c    4.466341
dtype: float64
a    84.653704
b     2.087463
c     4.466341
dtype: float64
9.35 ms ± 577 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
print(empyrical.calmar_ratio(returns['b']))
%timeit empyrical.calmar_ratio(big_returns[0])

print(returns['b'].vbt.returns.calmar_ratio())
%timeit big_returns[0].vbt.returns.calmar_ratio()

print(returns.vbt.returns.calmar_ratio())
%timeit big_returns.vbt.returns.calmar_ratio()

-1.2500000139698388
674 µs ± 61.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
-1.2500000139698388
341 µs ± 23.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a         NaN
b   -1.250000
c    0.000002
dtype: float64
20 ms ± 868 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
print(empyrical.omega_ratio(returns['c'], risk_free=0.01, required_return=0.1))
%timeit empyrical.omega_ratio(big_returns[0], risk_free=0.01, required_return=0.1)

print(returns['c'].vbt.returns.omega_ratio(risk_free=0.01, required_return=0.1))
%timeit big_returns[0].vbt.returns.omega_ratio(risk_free=0.01, required_return=0.1)

print(returns.vbt.returns.omega_ratio(risk_free=0.01, required_return=0.1))
print(returns.vbt.returns.omega_ratio(risk_free=[0.01, 0.02, 0.03], required_return=0.1))
print(returns.vbt.returns.omega_ratio(risk_free=0.01, required_return=[0.1, 0.2, 0.3]))
%timeit big_returns.vbt.returns.omega_ratio(risk_free=0.01, required_return=0.1)

1.7319528661672228
2.97 ms ± 168 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.7319528661672228
391 µs ± 31.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a         NaN
b    0.000000
c    1.731953
dtype: float64
a        NaN
b    0.00000
c    1.60973
dtype: float64
a         NaN
b    0.000000
c    1.727716
dtype: float64
17.2 ms ± 372 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
print(empyrical.sharpe_ratio(returns['a'], risk_free=0.01))
%timeit empyrical.sharpe_ratio(big_returns[0], risk_free=0.01)

print(returns['a'].vbt.returns.sharpe_ratio(risk_free=0.01))
%timeit big_returns[0].vbt.returns.sharpe_ratio(risk_free=0.01)

print(returns.vbt.returns.sharpe_ratio(risk_free=0.01))
print(returns.vbt.returns.sharpe_ratio(risk_free=[0.01, 0.02, 0.03]))
%timeit big_returns.vbt.returns.sharpe_ratio(risk_free=0.01)

24.139822936194918
459 µs ± 30 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
24.139822936194918
362 µs ± 26.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a    24.139823
b   -39.938439
c     3.517158
dtype: float64
a    24.139823
b   -41.145646
c     3.068159
dtype: float64
10.7 ms ± 393 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
print(empyrical.downside_risk(returns['b'], required_return=0.1))
%timeit empyrical.downside_risk(big_returns[0], required_return=0.1)

print(returns['b'].vbt.returns.downside_risk(required_return=0.1))
%timeit big_returns[0].vbt.returns.downside_risk(required_return=0.1)

print(returns.vbt.returns.downside_risk(required_return=0.1))
print(returns.vbt.returns.downside_risk(required_return=[0.1, 0.2, 0.3]))
%timeit big_returns.vbt.returns.downside_risk(required_return=0.1)

6.920801865722236
179 µs ± 3.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
6.920801865722236
326 µs ± 29.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a    0.000000
b    6.920802
c    5.874521
dtype: float64
a    0.000000
b    8.463303
c    8.098765
dtype: float64
11.3 ms ± 204 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
print(empyrical.sortino_ratio(returns['b'], required_return=0.1))
%timeit empyrical.sortino_ratio(big_returns[0], required_return=0.1)

print(returns['b'].vbt.returns.sortino_ratio(required_return=0.1))
%timeit big_returns[0].vbt.returns.sortino_ratio(required_return=0.1)

print(returns.vbt.returns.sortino_ratio(required_return=0.1))
print(returns.vbt.returns.sortino_ratio(required_return=[0.1, 0.2, 0.3]))
%timeit big_returns.vbt.returns.sortino_ratio(required_return=0.1)

-15.32336860018125
573 µs ± 54.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
-15.32336860018125
345 µs ± 24.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a          NaN
b   -15.323369
c     2.859808
dtype: float64
a          NaN
b   -15.508129
c    -4.148780
dtype: float64
14.5 ms ± 620 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
factor_returns = returns['a'] * np.random.uniform(0.8, 1.2, returns.shape[0])
big_factor_returns = big_returns[0] * np.random.uniform(0.8, 1.2, big_returns.shape[0])

In [23]:
print(empyrical.excess_sharpe(returns['a'], factor_returns))
%timeit empyrical.excess_sharpe(big_returns[0], factor_returns)

print(returns['a'].vbt.returns.information_ratio(factor_returns)) # will broadcast
%timeit big_returns[0].vbt.returns.information_ratio(big_factor_returns)

print(returns.vbt.returns.information_ratio(factor_returns))
%timeit big_returns.vbt.returns.information_ratio(big_factor_returns)

0.48218846060997633
1.24 ms ± 302 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
0.48218846060997633
569 µs ± 150 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a    0.482188
b   -4.896215
c   -0.656625
dtype: float64
12 ms ± 870 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
print(empyrical.beta(returns['a'], factor_returns))
%timeit empyrical.beta(big_returns[0], factor_returns)

print(returns['a'].vbt.returns.beta(factor_returns))
%timeit big_returns[0].vbt.returns.beta(big_factor_returns)

print(returns.vbt.returns.beta(factor_returns))
%timeit big_returns.vbt.returns.beta(big_factor_returns)

1.3214354004917297
2.94 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.3214354004917297
510 µs ± 43.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a    1.321435
b    0.426179
c    2.598135
dtype: float64
15.7 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
print(empyrical.alpha(returns['a'], factor_returns, risk_free=0.01))
%timeit empyrical.alpha(big_returns[0], factor_returns, risk_free=0.01)

print(returns['a'].vbt.returns.alpha(factor_returns, risk_free=0.01))
%timeit big_returns[0].vbt.returns.alpha(big_factor_returns, risk_free=0.01)

print(returns.vbt.returns.alpha(factor_returns, risk_free=0.01))
print(returns.vbt.returns.alpha(factor_returns, risk_free=[0.01, 0.02, 0.03]))
%timeit big_returns.vbt.returns.alpha(big_factor_returns, risk_free=0.01)

-0.9999999999998873
5.23 ms ± 709 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
-0.9999999999998873
562 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a   -1.0
b   -1.0
c   -1.0
dtype: float64
a   -1.0
b   -1.0
c   -1.0
dtype: float64
22.8 ms ± 905 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
print(empyrical.tail_ratio(returns['a']))
%timeit empyrical.tail_ratio(big_returns[0])

print(returns['a'].vbt.returns.tail_ratio())
%timeit big_returns[0].vbt.returns.tail_ratio()

print(returns.vbt.returns.tail_ratio())
%timeit big_returns.vbt.returns.tail_ratio()

3.5238094437960337
548 µs ± 31.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.5238094437960337
250 µs ± 33 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a    3.523809
b    0.436842
c    1.947368
dtype: float64
41.4 ms ± 1.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
print(empyrical.value_at_risk(returns.iloc[1:]['a'], cutoff=0.05))
%timeit empyrical.value_at_risk(big_returns[0], cutoff=0.05)

print(returns['a'].vbt.returns.value_at_risk(cutoff=0.05))
%timeit big_returns[0].vbt.returns.value_at_risk(cutoff=0.05)

print(returns.vbt.returns.value_at_risk(cutoff=0.05))
print(returns.vbt.returns.value_at_risk(cutoff=[0.05, 0.06, 0.07]))
%timeit big_returns.vbt.returns.value_at_risk(cutoff=0.05)

0.26250000596046447
301 µs ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
0.26250000596046447
296 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a    0.2625
b   -0.4750
c   -0.4750
dtype: float64
a    0.2625
b   -0.4700
c   -0.4650
dtype: float64
26.2 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
print(empyrical.conditional_value_at_risk(returns.iloc[1:]['a'], cutoff=0.05))
%timeit empyrical.conditional_value_at_risk(big_returns[0], cutoff=0.05)

print(returns['a'].vbt.returns.conditional_value_at_risk(cutoff=0.05))
%timeit big_returns[0].vbt.returns.conditional_value_at_risk(cutoff=0.05)

print(returns.vbt.returns.conditional_value_at_risk(cutoff=0.05))
print(returns.vbt.returns.conditional_value_at_risk(cutoff=[0.05, 0.06, 0.07]))
%timeit big_returns.vbt.returns.conditional_value_at_risk(cutoff=0.05)

0.25
121 µs ± 3.01 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
0.25
271 µs ± 15.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a    0.25
b   -0.50
c   -0.50
dtype: float64
a    0.25
b   -0.50
c   -0.50
dtype: float64
23.1 ms ± 1.92 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
print(empyrical.capture(returns['a'], factor_returns))
%timeit empyrical.capture(big_returns[0], big_factor_returns)

print(returns['a'].vbt.returns.capture(factor_returns))
%timeit big_returns[0].vbt.returns.capture(big_factor_returns)

print(returns.vbt.returns.capture(factor_returns))
%timeit big_returns.vbt.returns.capture(big_factor_returns)

56.23572072706983
778 µs ± 36 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
56.23572072706978
512 µs ± 30.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a    5.623572e+01
b   -3.326009e-34
c    4.995792e-40
dtype: float64
8.63 ms ± 321 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
print(empyrical.up_capture(returns['a'], factor_returns))
%timeit empyrical.up_capture(big_returns[0], big_factor_returns)

print(returns['a'].vbt.returns.up_capture(factor_returns))
%timeit big_returns[0].vbt.returns.up_capture(big_factor_returns)

print(returns.vbt.returns.up_capture(factor_returns))
%timeit big_returns.vbt.returns.up_capture(big_factor_returns)

153.99808910022426


/Users/olegpolakow/miniconda3/lib/python3.7/site-packages/empyrical/stats.py:1561: RuntimeWarning: invalid value encountered in double_scalars
  annual_return(factor_returns, period=period))


2.23 ms ± 200 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
153.99808910022426
508 µs ± 19.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a    1.539981e+02
b   -1.420379e-42
c    2.666830e-48
dtype: float64
10.7 ms ± 219 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
print(empyrical.down_capture(returns['a'], factor_returns))
%timeit empyrical.down_capture(big_returns[0], big_factor_returns)

print(returns['a'].vbt.returns.down_capture(factor_returns))
%timeit big_returns[0].vbt.returns.down_capture(big_factor_returns)

print(returns.vbt.returns.down_capture(factor_returns))
%timeit big_returns.vbt.returns.down_capture(big_factor_returns)

nan
2.52 ms ± 178 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
nan
566 µs ± 37.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
a   NaN
b   NaN
c   NaN
dtype: float64
10.9 ms ± 635 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
print(returns['a'].vbt.returns.skew())
%timeit big_returns[0].vbt.returns.skew()

print(returns.vbt.returns.skew())
%timeit big_returns.vbt.returns.skew() # very slow

0.8483206930595862
2.92 ms ± 254 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
a    0.848321
b   -0.615091
c    0.211684
dtype: float64
180 ms ± 16.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
print(returns['a'].vbt.returns.kurtosis())
%timeit big_returns[0].vbt.returns.kurtosis()

print(returns.vbt.returns.kurtosis())
%timeit big_returns.vbt.returns.kurtosis() # even slower

-0.9292307393753112
2.8 ms ± 206 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
a   -0.929231
b   -1.113971
c   -1.658436
dtype: float64
198 ms ± 3.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
print(returns.vbt.returns.drawdown())
print(returns.vbt.returns.drawdown(start_value=1.))

%timeit big_returns.vbt.returns.drawdown()

              a    b         c
2018-01-01  0.0  0.0  0.000000
2018-01-02  0.0 -0.2  0.000000
2018-01-03  0.0 -0.4  0.000000
2018-01-04  0.0 -0.6 -0.333333
2018-01-05  0.0 -0.8 -0.666667
              a    b         c
2018-01-01  0.0  0.0  0.000000
2018-01-02  0.0 -0.2  0.000000
2018-01-03  0.0 -0.4  0.000000
2018-01-04  0.0 -0.6 -0.333333
2018-01-05  0.0 -0.8 -0.666667
45 ms ± 5.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
print(returns.vbt.returns.drawdowns())
%timeit big_returns.vbt.returns.drawdowns()

24.9 ms ± 6.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
